# Diffraction Ring Based Detector Geometry Calibration

This notebook serves demonstrates how to use ipython based widgets to create a starting geometry from Power Based ring diffraction pattern.

In [ ]:
%matplotlib notebook
#This important for interactive plots 
import os
import warnings
import sys

from matplotlib import pyplot as plt
import matplotlib.patches as patches

parent = os.path.dirname(os.path.abspath(os.path.dirname('.')))
sys.path.insert(0, parent)
warnings.filterwarnings('ignore')
import numpy as np

import karabo_data as kd
from geoAssembler import CalibrateNb as Calibrate
from geoAssembler.geometry import AGIPDGeometry, LPDGeometry

# Goemetry using AGIPD Data


## Get a run dataset with ring pattern

The example *instrument* has a some runs with interesting data. Run 005 contains data with diffraction patterns from Lithium Titanium. First open this run with karabo-data and read the data


In [ ]:
run_dir = '/gpfs/exfel/exp/XMPL/201750/p700000/proc/r0005'
run = kd.RunDirectory(run_dir)
run.info()

## Define some parameters:
Detector distance to the sample, Beam wave length are certainly helpful variables that should be defined in the geometry file. Let's define those parameters:

In [ ]:
clen = 0.119 #Detector distance in m
energy = 10235 #Photon energy in eV I believe

In [ ]:
#Now construct the header of the geometry file from those properties
header = """data = /entry_1/data_1/data
;mask = /entry_1/data_1/mask

mask_good = 0x0
mask_bad = 0xffff

adu_per_eV = 0.0075  ; no idea
clen = {}  ; Camera length, aka detector distance
photon_energy = {} ;""".format(clen, energy)

## Read the train data

In [ ]:
tId, train_data = run.train_from_index(3)
tId

Use the ```stack_detector_data``` to create a big with all pulses for the selected train data. 

In [ ]:
train_array = kd.stack_detector_data(train_data, 'image.data', only='DET')
train_array.shape

Let's apply the ```sum``` function over a train rather than selecting single pulses. This can help to get a clearer signal, if necessary.

In [ ]:
data_array = np.sum(np.clip(train_array, 0, 2000), axis=0)
data_array.shape

### Start the geometry calibration without prior geometry knowledge
First lets assume that we have no prior knowledge about the detector geometry. We naively just assume that all modules equally spaced with 29px distance to each other.

In [ ]:
from geoAssembler import CalibrateNb as Calibrate
from geoAssembler import CalibrateNb as Calibrate
from geoAssembler.geometry import AGIPDGeometry, LPDGeometry
Calib =  Calibrate(data_array, geometry=None, vmin=0, vmax=1e5, figsize=(8,8), det='AGIPD')

Helper circles that guide the quadrant movement can be added by pushing the *Add circle* button. There can be multiple at a time. If there are different circles present, individual circles can be selected using the *Sel.* drop down menu. The radius can be adjusted with help of the *Radius* menu.

The quadrants can be moved by selecting a Quadrant from the *Quadrant* menu. The selected quadrant can be moved horizontally and vertically via the *Horizontal* and *Vertical* spin boxes.

The centre of the geometry can be retrieved with the ```centre``` attribute:

In [ ]:
Calib.centre

Finally the geometry can be saved by calling ```C.geom.write_geom```. This method gets the output filename and the 'header' that contains information about detector distance and photon energy.

In [ ]:
Calib.geom.write_geom('testing.geom', header)

The new geometry file could also be loaded for another round of refinement:

In [ ]:
C_new =  Calibrate(data_array, geometry=Calib.geom, vmin=0, vmax=1e5, figsize=(8,8), det='AGIPD')

# Goemetry using LPD Data

Geometry description for LPD data is not saved in *CFEL* format but the inhouse EuXFEL format. The EuXFEL format requires quadrant positions. 

## Get a run dataset with ring pattern

The example *instrument* has a some runs with interesting data. Run 007 contains data with diffraction patterns from Lithium Titanium. First open this run with karabo-data and read the data


In [ ]:
run_dir = '/gpfs/exfel/exp/XMPL/201750/p700000/proc/r0007'
run = kd.RunDirectory(run_dir)
run.info()

## Read the train data

In [ ]:
tId, train_data = run.train_from_index(3)
tId

Use the ```stack_detector_data``` to create a big with all pulses for the selected train data. 

In [ ]:
train_array = kd.stack_detector_data(train_data, 'image.data', only='DET')
train_array.shape

Let's apply the ```sum``` function over a train rather than selecting single pulses. This can help to get a clearer signal, if necessary.

In [ ]:
data_array = np.sum(np.clip(train_array, 0, 2000), axis=0)
data_array.shape

### Start the geometry calibration without prior geometry knowledge
From a users perspective the EuXFEL format is a little more complicated. Quadrant positions and a geometry file descrbing in positioning of the tails within the quadrants is needed. Like in the AGIPD case no geometry information can be passed to the calibration. In this case a pixel gap of 4 pixels per module and asic is assumed. 

Yet it is recommended to at least read a geometry file and use the fallback quadrant positions:

### Create a geometry object from an existing EuXFEL geometry file without prior know ledge of quad positions

Let's use the geometry file that comes with the documentation of karabo_data

In [ ]:
parent = os.path.dirname(os.path.dirname(kd.__file__))
geom_file = os.path.join(parent, 'docs', 'lpd_mar_18_axesfixed.h5')
geom = LPDGeometry.load(geom_file=geom_file, quad_pos=None)
Calib =  Calibrate(data_array, geometry=geom, vmin=0, vmax=100000, figsize=(8,8), det='LPD')

### Read the quadrant positions
Unlinke for CFEL geometry only the quadrant positions are of relevance they can be retreived using the 
```get_quad_pos``` method of the ```Calib.geom``` object.

In [ ]:
Calib.geom.quad_pos

The new geometry file could also be loaded for another round of refinement:

In [ ]:
Calib_New =  Calibrate(data_array, geometry=Calib.geom, vmin=0, vmax=100000, figsize=(8,8), det='LPD')